# Multi-series sample notebook
This notebook demonstrates loading multiple sample datasets (hypercar, gold, other sales), preparing them with the project's helper `prepare_df`, calling the `forecast_timeseries` pipeline, and plotting results. It runs inside the repository and adds the `backend` folder to `sys.path` so `app` can be imported.

In [ ]:
# Robustly find the repository root (walk upwards until backend/app/__init__.py is found) and add backend to sys.path
import sys
import os
import pathlib

# Start from the current working directory and walk up a few levels
p = pathlib.Path.cwd()
repo_root = None
for _ in range(8):
    candidate = p / 'backend' / 'app' / '__init__.py'
    if candidate.exists():
        repo_root = p
        break
    if p.parent == p:
        break
    p = p.parent
if repo_root is None:
    # Fallback: use notebook parent (useful when executed with different cwd)
    repo_root = pathlib.Path('..').resolve()
backend_path = (repo_root / 'backend').resolve()
if str(backend_path) not in sys.path:
    sys.path.insert(0, str(backend_path))
print('Added to sys.path:', backend_path)

# Imports (after adjusting sys.path)
import pandas as pd
import asyncio
import plotly.express as px
from app.core.forecasting import prepare_df, forecast_timeseries

In [ ]:
# Load sample CSVs from the project's data/ directory
hyper = pd.read_csv('../data/hypercar_sales.csv')
gold = pd.read_csv('../data/gold_sales.csv')
other = pd.read_csv('../data/other_sales.csv')

datasets = {'Hypercar': hyper, 'Gold': gold, 'Other': other}

In [ ]:
# Forecast multi-series demo (lightweight): run for hypercar models with short horizon
try:
    import asyncio
    from app.core.forecasting import forecast_multi_series
    print('Running forecast_multi_series on hypercar (periods=6) --- this should be quick')
    results = asyncio.run(forecast_multi_series(hyper, periods=6, series_col='model'))
    print('Series found:', list(results.keys()))
    # print a small sample of each forecast result
    for k, v in results.items():
        if isinstance(v, dict) and 'forecast_data' in v:
            ts = v['forecast_data']['timestamp'][:3]
            vals = v['forecast_data']['value'][:3]
            print(f'-- {k}: forecast sample (first 3):')
            for a, b in zip(ts, vals):
                print('   ', a, b)
        else:
            print(f'-- {k}: error or no forecast: {v}')
except Exception as e:
    print('forecast_multi_series demo failed:', e)

In [ ]:
# Demo cell: inspect hypercar dataset and prepare one series
try:
    print('Hypercar rows:', len(hyper))
    print(hyper.head().to_string())
    # group by model and show counts
    print('Counts per model:
', hyper.groupby('model').size())
    # prepare first model's series with prepare_df (safe: will handle common date/price columns)
    first_model = hyper['model'].unique()[0]
    print('Preparing series for model:', first_model)
    model_df = hyper[hyper['model'] == first_model][['date','price']].rename(columns={'date':'ds','price':'y'})
    std = prepare_df(model_df)
    print('Prepared shape:', std.shape)
    print(std.head().to_string())
except Exception as e:
    print('Demo cell failed:', e)

In [ ]:
results = {}
for name, df in datasets.items():
    print(f'Processing: {name}')
    d = prepare_df(df)
    # run the async pipeline with asyncio.run
    res = asyncio.run(forecast_timeseries(d, periods=12))
    results[name] = res

In [ ]:
# Plot historical + forecast for each series
for name, res in results.items():
    hist = pd.DataFrame({'ds': pd.to_datetime(res['historical_data']['timestamp']), 'y': res['historical_data']['value']})
    fc = pd.DataFrame({'ds': pd.to_datetime(res['forecast_data']['timestamp']), 'y': res['forecast_data']['value']})
    fig = px.line(pd.concat([hist.assign(type='history'), fc.assign(type='forecast')]), x='ds', y='y', color='type', title=f'{name} sales')
    fig.show()

Done — this notebook uses the project's forecasting pipeline and the sample datasets added under `data/`. If you run this notebook locally, make sure you have the dev dependencies installed (see `dev-requirements.txt`).